# Long / Short Signals based on Technical Indicators
Author: **Peeyush Sharma**; Feedback: **PSharma3@gmail.com**

This notebook captures some basic long / short signals based on some technical indicators. The core daily market data was retrieved from local catalog and was subsequently used to generate various technical indicators in CSV form for a group of equities. This analysis retrieves that data and generates a list of dates with opportunities for long / short decisions. Decisions are based on relative value of closing price of an equity over any given duration (say 90 days). We start with a  broad selection of dates and gradually add limiting factors to come with optimal days to purchase equities in a bear market scenario. Similar methods can be used for normal and bull market as well with some adjustments. 

This is still a very high-level decision making. A stock may be on a long bull run and the 90 day averages may still not capture full potential in the long run. That kind of analysis is more detailed and is not captured in this publicly shared notebook. 

In [2]:
import os
import pandas as pd
import numpy as np
import sqlalchemy
import mysql.connector
import sys
import os.path
import math
import shutil
import getpass
from scipy import stats
from os import path
from mysql.connector import errorcode
from sqlalchemy import MetaData, create_engine
from sqlalchemy.exc import IntegrityError
from datetime import datetime, timedelta
from sqlalchemy.exc import ResourceClosedError, ProgrammingError

pd.options.mode.chained_assignment = None 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import plotly.graph_objects as go
import plotly.express as px
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

In [3]:
BASE_DIR = 'C:\\Users\\pshar\\Dropbox\workspace\\HelloPython\\HistoricalMarketData\\TechnicalIndicators'
DURATIONS = (14, 30, 90, 200) # Roughly for bi-weekly, monthly, quarterly, and 200 days moving averages and other tech indicators

In [4]:
# Retrieve data for Technical Indicators from pre-calculated CSVs
def generateFilePath(symbol, date=None):
    if date is not None:
        strDate = datetime.strftime(date, '%Y%m%d')
        fileName = symbol.lower()+'_'+strDate+'.csv'
        filePath = os.path.join(BASE_DIR, fileName)
    else: 
        fileName = symbol.lower()+'.csv'
        filePath = os.path.join(BASE_DIR, fileName)
    if filePath is None:
            print('Could not find file for symbol:{}'.format(symbol))
    # print(filePath)
    return filePath, fileName

In [5]:
# Retrieve sample data for a stock for predictions. 
strDateTarget = '2012-01-03'
dtTarget = datetime.strptime(strDateTarget, '%Y-%m-%d')
# symbols = ['FB', 'MSFT', 'GOOGL', 'NFLX', 'AAPL', 'AMZN', 'WFC', 'TSLA', 'BAC', 'C', 'GS', 'JPM', 'MS', 'MRK', 'NKE']
symbol = 'AAPL'
# for symbol in symbols:
filePath, _ = generateFilePath(symbol)
if (filePath is not None):
    try: 
        dfrm = pd.read_csv(filePath)
        dfrm.set_index('date', inplace=True)
    except FileNotFoundError as e:
        print('Exception reading input data for symbol {}.'.format(symbol))
        print(e)        

A sneak peak at the data and it's statistical distribution

In [6]:
dfrm.tail(10)

,symbol,close,volume,mean_200,stddev_200,pcntleStdDevs_200,pcntleVolume_200,pcntleClosing_200,oscillator_200,accu_dist_200,...,stddev_30,accu_dist_90,bollingerLower_90,bollingerUpper_90,mean_90,oscillator_90,pcntleClosing_90,pcntleStdDevs_90,pcntleVolume_90,stddev_90
date,,,,,,,,,,,,,,,,,,,,,
2020-07-31,AAPL,425.04,93573867,314.190360,42.234888,2.158273,97.841727,100.000000,100.000000,4.232526e+07,...,12.949684,3.524123e+07,285.008510,409.967297,347.487903,100.000000,100.000000,4.838710,100.000000,31.239697
2020-08-03,AAPL,435.75,77037847,315.089270,43.791821,2.919708,91.970803,100.000000,100.000000,4.276345e+07,...,16.491433,3.594519e+07,284.984494,414.590990,349.787742,100.000000,100.000000,6.451613,98.387097,32.401624
2020-08-04,AAPL,438.66,43198092,315.990146,45.046456,3.649635,64.233577,100.000000,100.000000,4.288017e+07,...,19.674383,3.604617e+07,284.851525,419.275572,352.063548,100.000000,100.000000,8.064516,80.645161,33.606012
2020-08-05,AAPL,440.25,30497988,316.877153,46.280327,4.379562,31.386861,100.000000,100.000000,4.285130e+07,...,22.076236,3.596414e+07,284.793354,423.837614,354.315484,100.000000,100.000000,9.677419,43.548387,34.761065
2020-08-06,AAPL,455.61,50607225,317.882464,47.599413,5.072464,76.811594,100.000000,100.000000,4.290750e+07,...,25.573808,3.631581e+07,283.867623,429.662377,356.765000,100.000000,100.000000,11.290323,91.935484,36.448689
2020-08-07,AAPL,444.45,49511403,318.793022,48.626465,5.755396,71.942446,99.280576,95.173845,3.880886e+07,...,27.098901,3.105664e+07,283.721686,434.141217,358.931452,92.457421,98.387097,12.903226,85.483871,37.604883
2020-08-10,AAPL,450.91,53100856,319.774420,50.081788,0.724638,79.710145,99.275362,97.967480,4.150701e+07,...,29.401513,3.450125e+07,283.243768,439.003006,361.123387,96.823466,98.387097,1.612903,95.161290,38.939810
2020-08-11,AAPL,437.50,46975594,320.631449,51.074517,1.449275,66.666667,96.376812,92.168310,3.661637e+07,...,29.647012,2.790107e+07,283.980858,442.333336,363.157097,87.760205,91.935484,3.225806,77.419355,39.588119
2020-08-12,AAPL,452.04,41486205,321.600507,52.284475,2.173913,57.246377,99.275362,98.456149,4.211041e+07,...,30.641936,3.502844e+07,284.474640,446.497296,365.485968,97.586207,98.387097,4.838710,69.354839,40.505664


In [7]:
dfrm.describe()

,close,volume,mean_200,stddev_200,pcntleStdDevs_200,pcntleVolume_200,pcntleClosing_200,oscillator_200,accu_dist_200,bollingerUpper_200,...,stddev_30,accu_dist_90,bollingerLower_90,bollingerUpper_90,mean_90,oscillator_90,pcntleClosing_90,pcntleStdDevs_90,pcntleVolume_90,stddev_90
count,4644.000000,4.644000e+03,4644.000000,4643.000000,4643.000000,4644.000000,4644.000000,4643.000000,4.643000e+03,4643.000000,...,4643.000000,4.643000e+03,4643.000000,4643.000000,4644.000000,4643.000000,4644.000000,4643.000000,4644.000000,4643.000000
mean,74.454001,1.136306e+08,69.373770,6.192112,57.545249,48.679055,70.507260,69.234850,3.925548e+07,81.772580,...,2.344807,2.823780e+07,63.402763,80.423561,71.898032,65.463952,65.621777,51.443253,49.161085,4.255199
std,79.139156,9.825176e+07,71.348368,7.414130,35.951274,30.235379,32.307721,31.437621,9.089704e+07,84.829383,...,3.187027,9.544983e+07,64.883198,84.955248,74.699440,32.661717,33.420514,36.534677,30.458774,5.776387
min,0.937000,9.870000e+06,1.051388,0.046136,0.709220,0.709220,0.709220,0.000000,-2.937347e+08,1.150458,...,0.017894,-3.377561e+08,0.814706,1.081309,1.025778,0.000000,1.562500,1.562500,1.562500,0.021348
25%,10.863750,4.235500e+07,9.722113,1.210019,23.741007,21.582734,50.000000,50.735111,8.646812e+05,12.040535,...,0.441914,-1.496758e+07,8.827304,11.913734,10.349082,40.336908,38.095238,15.873016,21.822917,0.734695
50%,49.570000,8.425910e+07,47.732601,3.641235,62.411348,48.175182,84.671533,80.883830,3.428544e+07,52.385316,...,1.365994,2.642005e+07,45.425161,51.860426,49.002618,75.409836,77.777778,48.837209,48.437500,2.444019
75%,112.665000,1.530900e+08,108.334745,7.885517,94.964029,75.716874,97.794118,95.451930,9.714132e+07,124.312508,...,2.949970,8.413069e+07,102.955927,120.843090,112.286537,95.373192,96.721311,91.999488,76.562500,5.425143
max,460.040000,8.432600e+08,322.596475,53.401671,100.000000,100.000000,100.000000,100.000000,2.737654e+08,429.399816,...,32.123775,3.048555e+08,285.008510,451.099304,367.913387,100.000000,100.000000,100.000000,100.000000,41.592958


In [20]:
"""
Plot daily closing values and couple other technical indicators using plotly.express
# Note that this plot may not show up in some platforms
"""
# Plot works, but increases the size of GitHub upload by order of MBs. Commenting out before upload
"""
fig = px.line(dfrm, x=dfrm.index, y=['close', 'oscillator_30',  'mean_30'],  title='Time Series with Range Slider and Selectors')
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
"""
# fig.show()

'\nfig = px.line(dfrm, x=dfrm.index, y=[\'close\', \'oscillator_30\',  \'mean_30\'],  title=\'Time Series with Range Slider and Selectors\')\nfig.update_xaxes(\n    rangeslider_visible=True,\n    rangeselector=dict(\n        buttons=list([\n            dict(count=1, label="1m", step="month", stepmode="backward"),\n            dict(count=6, label="6m", step="month", stepmode="backward"),\n            dict(count=1, label="YTD", step="year", stepmode="todate"),\n            dict(count=1, label="1y", step="year", stepmode="backward"),\n            dict(step="all")\n        ])\n    )\n)\n'

## Strategies

### Buy Side Decisions based on a Waterfall Approach w/ Technical Indicators. 
We will start with picking dates where closing prices for AAPL were significantly lower relative to the 90 days range. For that, we identify days when AAPL closing prices was lower than the lower level of the moving 90 days Bollinger band. 

In [9]:
# Set sample duration for technical indicators
duration = 90

In [10]:
# Start simple. Identify days when closing price was lower than the lower band in Bollinger range
dates_lows_for_buy_ops_1 = [ date for date in dfrm.index if dfrm.loc[date, 'close'] < dfrm.loc[date, 'bollingerLower_'+str(duration)] ]
indices = [ datetime.strptime(strDate, '%Y-%m-%d') for strDate in dfrm.index ]
dfrm.index = indices
dfrm.index.name = 'date'
dates_lows_for_buy_ops_1[-20:]

['2018-11-13',
 '2018-11-14',
 '2018-11-15',
 '2018-11-16',
 '2018-11-19',
 '2018-11-20',
 '2018-11-21',
 '2018-11-23',
 '2018-11-26',
 '2018-11-27',
 '2018-12-07',
 '2018-12-21',
 '2018-12-24',
 '2020-03-12',
 '2020-03-16',
 '2020-03-17',
 '2020-03-18',
 '2020-03-19',
 '2020-03-20',
 '2020-03-23']

Let us dive deeper into return on a sample day. Pick a sample date from the response above and check whether closing price that day was indeed lower than preceding days. Take example of March 12th, 2020 just as NYSE and broader market were getting close to their lows COVID-19 pandemic. The next table captures daily closing prices of AAPL around 3/12/2020. 

In [11]:
strDateTarget = '2020-03-12'
# Check the closing values preceding it
dtTarget = datetime.strptime(strDateTarget, '%Y-%m-%d')
dfrm.loc[dtTarget - timedelta(days=15):dtTarget + timedelta(days=15),['close', 'volume']]

,close,volume
date,,
2020-02-26,292.65,49678431
2020-02-27,273.52,80151381
2020-02-28,273.36,106721230
2020-03-02,298.81,85349339
2020-03-03,289.32,79868852
2020-03-04,302.74,54794568
2020-03-05,292.92,46893219
2020-03-06,289.03,56544246
2020-03-09,266.17,71686208


So 2020-03-12 was indeed a good opportunity to buy AAPL. That said, as it turned out, it was a local minima. The AAPL price continued to go down over next few days after 2020-03-12. The question is can we avoid purchasing in the local minima and instead wait out to get closer to a minima over a broader period. We will try to make the decision more efficient.

Let us include some other technical indicators beyond Bollinger bands. Let us look for days when percentile closing value is in lowest 10% but sale volume is in highest 90%. Go long on first such day itself. We must note however that this strategy can only be applied for companies with robust cash flows and balance sheet. For companies with volatile earnings and weaker balance sheets, a high interest at very low prices may simply be the start of a long sale cycle. 

In [12]:
dates_lows_for_buy_ops_2 = [ date for date in dfrm.index if dfrm.loc[date, 'pcntleClosing_'+str(duration)] < 10 and dfrm.loc[date, 'pcntleVolume_'+str(duration)] > 90]
dates_lows_for_buy_ops_2[-20:]

[Timestamp('2018-02-07 00:00:00'),
 Timestamp('2018-02-08 00:00:00'),
 Timestamp('2018-02-09 00:00:00'),
 Timestamp('2018-02-12 00:00:00'),
 Timestamp('2018-11-02 00:00:00'),
 Timestamp('2018-11-05 00:00:00'),
 Timestamp('2018-11-12 00:00:00'),
 Timestamp('2018-11-14 00:00:00'),
 Timestamp('2018-11-20 00:00:00'),
 Timestamp('2018-12-10 00:00:00'),
 Timestamp('2018-12-20 00:00:00'),
 Timestamp('2018-12-21 00:00:00'),
 Timestamp('2019-01-03 00:00:00'),
 Timestamp('2020-03-09 00:00:00'),
 Timestamp('2020-03-12 00:00:00'),
 Timestamp('2020-03-13 00:00:00'),
 Timestamp('2020-03-16 00:00:00'),
 Timestamp('2020-03-17 00:00:00'),
 Timestamp('2020-03-20 00:00:00'),
 Timestamp('2020-03-23 00:00:00')]

We were not able to reduce count with the combination of percentile closing and percentile volume. Let us try with stochastic oscillators.

In [13]:
dates_lows_for_buy_ops_3 = [ date for date in dfrm.index if dfrm.loc[date, 'oscillator_'+str(duration)] < 25 and dfrm.loc[date, 'pcntleVolume_'+str(duration)] > 90 ]
dates_lows_for_buy_ops_3[-20:]

[Timestamp('2018-02-07 00:00:00'),
 Timestamp('2018-02-08 00:00:00'),
 Timestamp('2018-02-09 00:00:00'),
 Timestamp('2018-11-02 00:00:00'),
 Timestamp('2018-11-05 00:00:00'),
 Timestamp('2018-11-12 00:00:00'),
 Timestamp('2018-11-14 00:00:00'),
 Timestamp('2018-11-20 00:00:00'),
 Timestamp('2018-12-10 00:00:00'),
 Timestamp('2018-12-20 00:00:00'),
 Timestamp('2018-12-21 00:00:00'),
 Timestamp('2019-01-03 00:00:00'),
 Timestamp('2020-02-27 00:00:00'),
 Timestamp('2020-02-28 00:00:00'),
 Timestamp('2020-03-09 00:00:00'),
 Timestamp('2020-03-12 00:00:00'),
 Timestamp('2020-03-16 00:00:00'),
 Timestamp('2020-03-17 00:00:00'),
 Timestamp('2020-03-20 00:00:00'),
 Timestamp('2020-03-23 00:00:00')]

Not significant difference either. March 2020 still shows a lot of days with buy options. Let us introduce average of standard deviation and include that in our filter as well. 

In [14]:
dates_lows_for_buy_ops_4 = [ date for date in dfrm.index if dfrm.loc[date, 'oscillator_'+str(duration)] < 25 and dfrm.loc[date, 'pcntleVolume_'+str(duration)] > 90 and dfrm.loc[date, 'pcntleStdDevs_'+str(duration)] > 90]
dates_lows_for_buy_ops_4[-20:]

[Timestamp('2015-08-05 00:00:00'),
 Timestamp('2015-08-11 00:00:00'),
 Timestamp('2015-08-12 00:00:00'),
 Timestamp('2015-08-21 00:00:00'),
 Timestamp('2015-08-24 00:00:00'),
 Timestamp('2015-08-25 00:00:00'),
 Timestamp('2016-01-07 00:00:00'),
 Timestamp('2016-01-08 00:00:00'),
 Timestamp('2016-01-15 00:00:00'),
 Timestamp('2016-01-20 00:00:00'),
 Timestamp('2016-01-26 00:00:00'),
 Timestamp('2016-01-27 00:00:00'),
 Timestamp('2016-06-17 00:00:00'),
 Timestamp('2016-06-24 00:00:00'),
 Timestamp('2018-12-10 00:00:00'),
 Timestamp('2018-12-20 00:00:00'),
 Timestamp('2018-12-21 00:00:00'),
 Timestamp('2019-01-03 00:00:00'),
 Timestamp('2020-03-20 00:00:00'),
 Timestamp('2020-03-23 00:00:00')]

Now, we only get March 20th and 23rd as potential buying opportunities for AAPL. So, in all, this was a gradual filtering of optimal purchase days. This example focused at the peak of COVID-19 impact on financial markets. Generally speaking filtering upto the level of 'dates_lows_for_buy_ops_4' will not yield any buying opportunity in normal market conditions. An asset manager needs to start with earlier levels and then apply her assessment of economic conditions to purchase at the optimal time of her choice. 

### Sale Side Decisions based on a Waterfall Approach w/ Technical Indicators

In [15]:
dates_highs_for_sale_ops_1 = [ date for date in dfrm.index if dfrm.loc[date, 'close'] > dfrm.loc[date, 'bollingerUpper_'+str(duration)] ]
dates_highs_for_sale_ops_1[-20:]

[Timestamp('2019-11-13 00:00:00'),
 Timestamp('2020-01-02 00:00:00'),
 Timestamp('2020-01-06 00:00:00'),
 Timestamp('2020-01-08 00:00:00'),
 Timestamp('2020-01-09 00:00:00'),
 Timestamp('2020-01-10 00:00:00'),
 Timestamp('2020-01-13 00:00:00'),
 Timestamp('2020-01-14 00:00:00'),
 Timestamp('2020-01-16 00:00:00'),
 Timestamp('2020-01-17 00:00:00'),
 Timestamp('2020-06-10 00:00:00'),
 Timestamp('2020-07-31 00:00:00'),
 Timestamp('2020-08-03 00:00:00'),
 Timestamp('2020-08-04 00:00:00'),
 Timestamp('2020-08-05 00:00:00'),
 Timestamp('2020-08-06 00:00:00'),
 Timestamp('2020-08-07 00:00:00'),
 Timestamp('2020-08-10 00:00:00'),
 Timestamp('2020-08-12 00:00:00'),
 Timestamp('2020-08-13 00:00:00')]

We right away see that there was no sale opportunity at the peak of COVID-19. The first opportunity after that seems to have come around June 10th. Let us check returns around that date. 

In [16]:
strDateTarget = '2020-06-10'
# Check the closing values around the target date 
dtTarget = datetime.strptime(strDateTarget, '%Y-%m-%d')
dfrm.loc[dtTarget - timedelta(days=15):dtTarget + timedelta(days=15),['close', 'volume']]

,close,volume
date,,
2020-05-26,316.73,31380454
2020-05-27,318.11,28236274
2020-05-28,318.25,33449103
2020-05-29,317.94,38399532
2020-06-01,321.85,20254653
2020-06-02,323.34,21910704
2020-06-03,325.12,26122804
2020-06-04,322.32,21890091
2020-06-05,331.50,34312550


Seems like June 10th was a resonably local high for sale opportunity. We know that afterward in late summer 2020, the large tech sector had a strong showing. That presented with further several sales opportunities for AAPL. Let us further see if we can identify a more narrow sale opportunity in early to mid-August when AAPL hit several peaks. Similar to opportunities on buy side, we will now add percentile closing and volume factors. 

In [17]:
dates_highs_for_sale_ops_2 = [ date for date in dfrm.index if dfrm.loc[date, 'pcntleClosing_'+str(duration)] > 90 and dfrm.loc[date, 'pcntleVolume_'+str(duration)] > 90]
dates_highs_for_sale_ops_2[-20:]

[Timestamp('2019-07-31 00:00:00'),
 Timestamp('2019-08-13 00:00:00'),
 Timestamp('2019-09-11 00:00:00'),
 Timestamp('2019-12-20 00:00:00'),
 Timestamp('2019-12-27 00:00:00'),
 Timestamp('2019-12-30 00:00:00'),
 Timestamp('2020-01-03 00:00:00'),
 Timestamp('2020-01-09 00:00:00'),
 Timestamp('2020-01-14 00:00:00'),
 Timestamp('2020-01-24 00:00:00'),
 Timestamp('2020-01-28 00:00:00'),
 Timestamp('2020-01-29 00:00:00'),
 Timestamp('2020-06-19 00:00:00'),
 Timestamp('2020-06-23 00:00:00'),
 Timestamp('2020-06-26 00:00:00'),
 Timestamp('2020-07-31 00:00:00'),
 Timestamp('2020-08-03 00:00:00'),
 Timestamp('2020-08-06 00:00:00'),
 Timestamp('2020-08-10 00:00:00'),
 Timestamp('2020-08-13 00:00:00')]

And we did see a somewhat more narrow range indeed for early to mid-August time-frame. Let us narrow it down further. 

In [18]:
dates_lows_for_sale_ops_3 = [ date for date in dfrm.index if dfrm.loc[date, 'oscillator_'+str(duration)] > 75 and dfrm.loc[date, 'pcntleVolume_'+str(duration)] > 90 ]
dates_lows_for_sale_ops_4 = [ date for date in dfrm.index if dfrm.loc[date, 'oscillator_'+str(duration)] > 75 and dfrm.loc[date, 'pcntleVolume_'+str(duration)] > 90 and dfrm.loc[date, 'pcntleStdDevs_'+str(duration)] > 90]
dates_lows_for_sale_ops_3[-20:]
dates_lows_for_sale_ops_4[-20:]

[Timestamp('2017-09-13 00:00:00'),
 Timestamp('2018-05-02 00:00:00'),
 Timestamp('2018-05-04 00:00:00'),
 Timestamp('2018-06-15 00:00:00'),
 Timestamp('2018-09-10 00:00:00'),
 Timestamp('2018-09-11 00:00:00'),
 Timestamp('2018-09-12 00:00:00'),
 Timestamp('2018-09-13 00:00:00'),
 Timestamp('2018-09-17 00:00:00'),
 Timestamp('2018-09-21 00:00:00'),
 Timestamp('2020-01-14 00:00:00'),
 Timestamp('2020-01-24 00:00:00'),
 Timestamp('2020-01-27 00:00:00'),
 Timestamp('2020-01-28 00:00:00'),
 Timestamp('2020-01-29 00:00:00'),
 Timestamp('2020-01-31 00:00:00'),
 Timestamp('2020-02-03 00:00:00'),
 Timestamp('2020-06-19 00:00:00'),
 Timestamp('2020-06-23 00:00:00'),
 Timestamp('2020-06-26 00:00:00')]

The mean of standard deviation did not help much for long decisions. This is likely because AAPL had a great run with not as much volatility to increase std deviation significantly. 

## Limitations and TODO Items: 
There are several limitations for this kind of analysis. Some of the more noteworthy ones are:
- This type of analysis is focusing only on market price during a given time. Market price may not capture the full intrinsic potential of future earnings of a company. This analysis is strictly limited to the range within price movements occurs and needs to be complimented with financial analysis of returns and potential of future earnings for a full picture. 
- In a low volatility market, the filters used here may either list a lot of days or hardly any to go long / short. The criteria may need to be relaxed or hardened going by prevailing market conditions. 
- The same technical indicator and analysis may not equally apply across stocks and sectors. 
